# NECS tutorial - Basic network traffic analysis using Time Series Analysis

For analysis we will use subsequence of the IP flows of HTTPS data from our network NETMONLAB (Network Monitoring Laboratory)

In [ ]:
!pip3 install numpy
!pip3 install pandas
!pip3 install matplotlib

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import time
import matplotlib.pyplot as plt

Load data

In [ ]:
df = pd.read_csv("necs_data.csv")

In [ ]:
df

In [ ]:
df["TIME_FIRST"] = df["TIME_FIRST"].apply(datetime.utcfromtimestamp)
df

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
ax.scatter(df.TIME_FIRST, df.PACKETS)
plt.xlabel("Time")
plt.ylabel("Number of packets")
plt.show()

Aggregate network traffic per hour

In [ ]:
df_agg = df.resample('H', on='TIME_FIRST')[["PACKETS"]].sum()
df_agg

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
df_agg.PACKETS.plot(legend=True, fig=fig)
plt.xlabel("Time")
plt.ylabel("Number of packets")
plt.show()

# Time Series Forecasting

Network traffic forecasting provides critical information for network management, resource allocation, and attack detection. We will forecast by using the basic ARIMA model.

### ARIMA model 
 
ARIMA is an extension of the ARMA model (Autoregressive Moving average), which also allows use on non-stationary time series. We write as: 

$ARIMA(p,d,q)$ 

Where $p$ is the order of the AR component and expresses how many time intervals back this component of the model "looks", $d$ is the order of the integration component and means the number of consecutive times the difference is applied (removal of trend, $d = 1$, or/and seasonality, $d = 2$), and $q$ is the order of the MA component and expresses from how many time intervals in the past the errors in the model are applied.

To capture the seasonality of time series, the basic model can be supplemented with an ARIMA model of the seasonal component, the parameters of which are marked with capital letters and given in additional brackets with $M$ that represents periodicity, in total: 

$ARIMA(p, d, q)(P, D, Q, M)$

The $(P, D, Q, M)$ order refers to the seasonal component of the model for the Auto Regressive parameters, differences, Moving Average parameters, and periodicity:

- $D$ indicates the integration order of the seasonal process (the number of transformations needed to make stationary the time series)
- $P$ indicates the Auto Regressive order for the seasonal component
- $Q$ indicated the Moving Average order for the seasonal component
- $M$ indicates the periodicity, i.e., the number of periods in season, such as 12 for monthly data.

ARIMA models are estimated using the so-called Box–Jenkins method, proposed by George Box and Gwilym Jenkins. It has three steps:

1. Model order identification and selection. This part of the analysis is to find out which values ​​of the orders $p$, $d$, $q$ or $P$, $D$, $Q$ are to be set. Here, the analysis of autocorrelations and partial autocorrelations of the examined time series is used.
2. Estimation of regression coefficients, usually by the method of maximum likelihood
3. Testing the model, especially the stationarity of its residuals.

In [ ]:
!pip3 install pmdarima
!pip3 install statsmodels

In [ ]:
import pmdarima as pm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, arma_order_select_ic
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
df_small = df[df.TIME_FIRST < "11.20.2022"]
df_small = df_small.resample('H', on='TIME_FIRST')[["PACKETS"]].sum()
display(df_small, df_agg)

#### Selecting $(p,d,q)$

Find the $(p,d,q)$ by auto_arima:

In [ ]:
stepwise_fit = pm.auto_arima(df_small, start_p=1, start_q=1,
                             max_p=3, max_q=3, m=7,
                             start_P=0, seasonal=True,
                             d=1, trace=True, stepwise=False,
                             information_criterion='aic',
                             error_action="ignore")

So the order parameter of ARIMA model will be $(0,1,1)$

#### Selecting $(P,D,Q,M)$
Next we want to set seasonal order parameter of ARIMA model. 

##### $M$ order
First we must find seasonality by seasonal decompose:

In [ ]:
result = seasonal_decompose(df_small.PACKETS, model='additive')
resid_acf = acf(result.resid, nlags=10, missing='drop')
sum_of_squares_resid_acf = np.sum(resid_acf**2)
print('Sum of Squares of ACF Residuals:', sum_of_squares_resid_acf)
result.plot()
plt.show()

In [ ]:
result._seasonal[:25]

So the seasonality is 24 hours. That means $M = 24$.

##### $D$ order
Check stationarity of time series. If time series is stationary, then D order is setted to 0.

In [ ]:
def check_stationarity(ts):
    dftest = adfuller(ts)
    adf = dftest[0]
    pvalue = dftest[1]
    critical_value = dftest[4]['5%']
    if (pvalue < 0.05) and (adf < critical_value):
        print('The series is stationary')
    else:
        print('The series is NOT stationary')

In [ ]:
seasonal = result.seasonal
check_stationarity(seasonal)

The searies is stationary so we can set $D = 0$

##### $P$ order
The value of $P$ order can be extracted by looking at the Partial Autocorrelation (PACF) graph of the seasonal component. PACF can be imagined as the correlation between the series and its lag, after excluding the contributions from the intermediate lags.

In [ ]:
plot_pacf(seasonal, lags=24)
plt.show()

We select P order as first lag of PACF that is out of confidental interval. So the $P = 1$

##### $Q$ order

The $Q$ order can be calculated from the Autocorrelation (ACF) plot. Autocorrelation is the correlation of a single time series with a lagged copy of itself.

In [ ]:
plot_acf(seasonal, lags=24)
plt.show()

From ACF we select $Q$ order as the lag that is out of confidental interval before first in confidental interval. That means $Q = 4$

So the best seasonal_order $(P,D,Q,M) = (1,0,4,24)$ 

##### Forecast using $ARIMA(0,1,1)(1,0,4,24)$

In [ ]:
model = ARIMA(df_small.PACKETS, order=(0,1,1), seasonal_order=(1,0,4,24))
res = model.fit()

fcast_horizon = df_agg.PACKETS.index[-1]
preds = res.get_prediction(end=fcast_horizon)

ci = preds.conf_int()
ci.loc[ci["lower PACKETS"] < 0,"lower PACKETS"] = 0

fig = ci.plot(color='grey', figsize=(12, 5))
df_agg.PACKETS.plot(label='data - hidden', marker='.')
res.data.orig_endog.plot(label='data - modelled', marker='.', fig=fig)
preds.predicted_mean.plot(label='predicted', fig=fig)
plt.legend()
plt.ylim((-1000,300000))
plt.xlim(("11.15.2022","11.28.2022"))
plt.show()

We can see that the prediction accuracy gradually deteriorates